In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import string
import matplotlib.ticker as ticker
from sklearn import preprocessing
import nltk
nltk.download('stopwords')
nltk.download('punkt')
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#downloading the NLTK package
!conda install -c conda-forge nltk --yes

Solving environment: - 

<b>Loading the data from the CSV file</b>

In [2]:
df = pd.read_csv('train_data.csv')
df.head()

,text,id
0,"Keep your gloves, hats, coats and jackets toge...",122885
1,The Home Dynamix Serendipity Ivory 5 ft. 2 in....,188958
2,The Bosch 18-Volt lithium-ion line of Cordless...,146065
3,Restore your Porter-Cable sander or polisher t...,165138
4,The SPIKECUBE Surge Suppressor from Tripp Lite...,185565


Checking the dimensions of df

In [3]:
df.shape

(41569, 2)

<b> Tokenization and Pre-processing</b>

In [4]:
#Tokenization using the NLTK package
from nltk.tokenize import sent_tokenize, word_tokenize

#Converting everything in the dataframe to lowercase
df.text = df.text.apply(lambda x: x.lower())

#Deleting the punctuation from the text column
df.text = df.text.apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))

#Deleting the digits from the text column
df.text = df.text.apply(lambda x: x.translate(str.maketrans('','',string.digits)))
df.head()

#Tokenizing the words in the dataframe
#df.text = df.text.astype(str)
df.text = df.text.apply(lambda x: nltk.word_tokenize(x))

df.head()

,text,id
0,"[keep, your, gloves, hats, coats, and, jackets...",122885
1,"[the, home, dynamix, serendipity, ivory, ft, i...",188958
2,"[the, bosch, volt, lithiumion, line, of, cordl...",146065
3,"[restore, your, portercable, sander, or, polis...",165138
4,"[the, spikecube, surge, suppressor, from, trip...",185565


<b>Removing the Stopwords</b>

In [5]:
from nltk.corpus import stopwords

# Bring in the default English NLTK stop words
stoplist = stopwords.words('english')

# Define additional stopwords in a string
additional_stopwords = """Home Depot Protection Plan Product Warranty"""

# Split the the additional stopwords string on each word and then add those words to the NLTK stopwords list
stoplist += additional_stopwords.split()

df.text = df.text.apply(lambda x: [item for item in x if item not in stoplist])
df.head()

,text,id
0,"[keep, gloves, hats, coats, jackets, together,...",122885
1,"[home, dynamix, serendipity, ivory, ft, x, ft,...",188958
2,"[bosch, volt, lithiumion, line, cordless, reci...",146065
3,"[restore, portercable, sander, polisher, origi...",165138
4,"[spikecube, surge, suppressor, tripp, lite, di...",185565


Load the train_label dataset and merge it with train_data using the id column

In [6]:
#Load the train_label.csv into a dataframe
df1 = pd.read_csv('train_label.csv')
df1.head()

,id,label
0,100003,Shape
1,100004,Voltage (volts)
2,100004,Wattage (watts)
3,100006,Wattage (watts)
4,100007,ENERGY STAR Certified


Check the dimensions of df1 dataframe

In [7]:
print('df1 has',df1.shape[0],'rows and',df1.shape[1],'colums')

df1 has 75170 rows and 2 colums


<b>Let's group the labels for the same id together</b>

In [8]:
df2 = df1.groupby('id').agg(lambda x: x.tolist())
df2.head()

,label
id,
100003,[Shape]
100004,"[Voltage (volts), Wattage (watts)]"
100006,[Wattage (watts)]
100007,[ENERGY STAR Certified]
100008,"[Finish, Indoor/Outdoor, Package Quantity]"


<b>Merging the dataframes df and df1 based on the 'id' column</b>

In [9]:
df_merged = pd.merge(df,df2, on='id', how='left')
df_merged.head()

,text,id,label
0,"[keep, gloves, hats, coats, jackets, together,...",122885,"[Assembly Required, Finish, Hardware Included]"
1,"[home, dynamix, serendipity, ivory, ft, x, ft,...",188958,"[Commercial / Residential, Features, Flooring ..."
2,"[bosch, volt, lithiumion, line, cordless, reci...",146065,"[Tools Product Type, Voltage (volts)]"
3,"[restore, portercable, sander, polisher, origi...",165138,[Tools Product Type]
4,"[spikecube, surge, suppressor, tripp, lite, di...",185565,"[Commercial / Residential, Package Quantity]"


In [10]:
#Finding the max legth of label
max(map(len,df_merged['label']))

6

In [11]:
column=['id','label1','label2','label3','label4','label5','label6']
df_labels = pd.DataFrame(columns=column)

df_merged = df_merged.reindex( columns = df_merged.columns.tolist() + ['label1','label2','label3','label4','label5','label6'])

In [ ]:
for index, row in df_merged.iterrows():
    labelList=row['label']
    #if index<2:
        #print(labelList)
    for n in range(6):
        labelname='label'+ str(n+1)
        if len(labelList)>n:
            df_merged.iloc[index,n+3]=[labelList[n]]
            #row[labelname]=[labelList[n]]
        else:
            df_merged.iloc[index,n+3]=['0']
            #row[labelname]=['0']
        #if index<2:
            #print(labelname)
            #print(row[labelname])
            #print(df_merged.iloc[index,n+3])
    #df_labels.append(t,ignore_index=True)

#df_updated=pd.merge(df_merged,df_labels,on='id',how='left')    
#df_updated.head()
df_merged.head()

<b> Visualizing the data for insights</b>

In [13]:
#installing the seaborn package
!conda install -c anaconda seaborn -y

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - seaborn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libssh2-1.8.0              |       h1ba5d50_4         233 KB  anaconda
    libcurl-7.63.0             |    h20c2e04_1000         550 KB  anaconda
    pycurl-7.43.0.2            |   py36h1ba5d50_0         185 KB  anaconda
    qt-5.9.7                   |       h5867ecd_1        85.9 MB  anaconda
    krb5-1.16.1                |       h173b8e3_7         1.4 MB  anaconda
    ca-certificates-2018.12.5  |                0         123 KB  anaconda
    conda-4.6.1                |           py36_0         1.7 MB  anaconda
    cryptography-2.4.2         |   py36h1ba5d50_0         618 KB  anaconda
    certifi-2018.11.29         |           py36_0         146 KB  anaconda
    curl-7.63.0                |    hbc83047

<b>Preparing the dataframe for model fitting and transformation</b>

In [144]:
df_merged = df_merged.drop_duplicates(['id','label'])
df_new = df_merged.pivot(index='id', columns='label')
df_new.columns = df_new.columns.droplevel(0)
df_new.head()

label,Assembly Required,Color,Commercial / Residential,ENERGY STAR Certified,Features,Finish,Flooring Product Type,Hardware Included,Included,Indoor/Outdoor,Package Quantity,Shape,Tools Product Type,Voltage (volts),Wattage (watts)
id,,,,,,,,,,,,,,,
100003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[classic, architecture, meets, contemporary, d...",NaN,NaN,NaN
100004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[grape, solar, watt, polycrystalline, pv, sola...","[grape, solar, watt, polycrystalline, pv, sola..."
100006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[achieving, delicious, results, almost, effort..."
100007,NaN,NaN,NaN,"[quantum, adjustable, light, led, black, emerg...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100008,NaN,NaN,NaN,NaN,NaN,"[teks, x, zincplated, steel, washerhead, hex, ...",NaN,NaN,NaN,"[teks, x, zincplated, steel, washerhead, hex, ...","[teks, x, zincplated, steel, washerhead, hex, ...",NaN,NaN,NaN,NaN


<b>Using One hot encoding and converting the categorical variables to binary values</b>

In [145]:
#Converting all the NaN's to zeroes and Non NaN's to 1
df_onehot = df_new.notnull().astype('int')
df_onehot.head()

label,Assembly Required,Color,Commercial / Residential,ENERGY STAR Certified,Features,Finish,Flooring Product Type,Hardware Included,Included,Indoor/Outdoor,Package Quantity,Shape,Tools Product Type,Voltage (volts),Wattage (watts)
id,,,,,,,,,,,,,,,
100003,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
100004,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
100006,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
100007,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
100008,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0


<h3>Feature Selection</h3>

In [146]:
X = df_onehot
X[0:5]
X.shape

(41569, 15)

Defining the labels

In [148]:
y = df1['label'].values
y[0:5]
y.shape

(75170,)

<h3>Normalize data</h3>

Data Standardization give data zero mean and unit variance (technically should be done after train test split )

In [149]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

/home/jupyterlab/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


array([[-0.3381727 , -0.34863522, -0.4448037 , -0.43087488, -0.31815705,
        -0.30801276, -0.34454215, -0.38930662, -0.34689686, -0.54126745,
        -0.36955707,  3.29236189, -0.35738364, -0.35652703, -0.32249393],
       [-0.3381727 , -0.34863522, -0.4448037 , -0.43087488, -0.31815705,
        -0.30801276, -0.34454215, -0.38930662, -0.34689686, -0.54126745,
        -0.36955707, -0.30373332, -0.35738364,  2.80483645,  3.10083352],
       [-0.3381727 , -0.34863522, -0.4448037 , -0.43087488, -0.31815705,
        -0.30801276, -0.34454215, -0.38930662, -0.34689686, -0.54126745,
        -0.36955707, -0.30373332, -0.35738364, -0.35652703,  3.10083352],
       [-0.3381727 , -0.34863522, -0.4448037 ,  2.32085934, -0.31815705,
        -0.30801276, -0.34454215, -0.38930662, -0.34689686, -0.54126745,
        -0.36955707, -0.30373332, -0.35738364, -0.35652703, -0.32249393],
       [-0.3381727 , -0.34863522, -0.4448037 , -0.43087488, -0.31815705,
         3.24661871, -0.34454215, -0.38930662, 

<h2>Classification</h2>

I'm going to use the training set to build a model. I'll then use the test set to report the accuracy of the model using the following algorithms:

* K Nearest Neighbor(KNN)
* Decision Tree
* Logistic Regression

I'll do the model training using the train dataset provided. Next step is to load the test data and apply the noise removal and pre-processing to it so that it matches the train data. It'll then be fed to the ML algorithms to predict the labels for the text provided.
The performance of the model is then evaluated and a final dataframe consisting of id's as the index and labels as column headers with the text label prediction probability updated as the values.